<a href="https://colab.research.google.com/github/clemvnt/training-datamining-mds/blob/issue5/20200424_05_BERLIN_final1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Goal**

Top 10 richest regions & countries in GDP per capita (wealth created per habitant) and in GDP current (total wealth created by the country)

**Data**

GDP CURRENT & GDP PER CAPITA by countries, agregated by region

**Sources**

* World Bank national accounts data
* OECD National Accounts data files 

**Notes**

The top 10 for GDP current is including the G8, should the European Union be included in this ranking, it would come up 2nd biggest economy after the USA. 

In the top 10 for GDP per capita, the ranking include smaller countries, only the USA remains in this ranking from the GDP current ranking.


**Pitch**

https://drive.google.com/file/d/1wGo9aI6mXS_2AbmnNlLoSJ9bGxkXmDhq/view

**Step 1 : Import dependencies**

In [0]:
import pandas as pd
from pandas_datareader import wb
import plotly.graph_objects as go

pd.options.display.float_format = '{: .0f}'.format

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


**Step 2 : Get the association between the country and the region**



In [0]:
countries = wb.get_countries()
countries = countries[['name', 'region']]
countries

,name,region
0,Aruba,Latin America & Caribbean
1,Afghanistan,South Asia
2,Africa,Aggregates
3,Angola,Sub-Saharan Africa
4,Albania,Europe & Central Asia
...,...,...
299,Sub-Saharan Africa excluding South Africa and ...,Aggregates
300,"Yemen, Rep.",Middle East & North Africa
301,South Africa,Sub-Saharan Africa
302,Zambia,Sub-Saharan Africa


**Step 3 : Get indicators**


In [0]:
indicators = wb.download(indicator=['NY.GDP.PCAP.CD', 'NY.GDP.MKTP.CD'], country='all', start=2018, end=2018)
indicators = indicators.reset_index()
indicators = indicators[['country', 'NY.GDP.PCAP.CD', 'NY.GDP.MKTP.CD']]
indicators.columns = ['country', 'GDP_PER_CAPITA', 'CURRENT_GDP']
indicators

,country,GDP_PER_CAPITA,CURRENT_GDP
0,Arab World,6609,2774314967156
1,Caribbean small states,9991,73523538155
2,Central Europe and the Baltics,15929,1632912822080
3,Early-demographic dividend,3582,11638859227437
4,East Asia & Pacific,11143,25942413437360
...,...,...,...
259,Virgin Islands (U.S.),nan,nan
260,West Bank and Gaza,3199,14615900000
261,"Yemen, Rep.",944,26914402224
262,Zambia,1540,26720073436


**Step 4 : Format a master table**

1. Associate countries with regions
1. Clean up the data
1. Group rows by columns 

In [0]:
master_table = pd.merge(indicators, countries, left_on='country', right_on='name')

master_table = master_table[master_table['region'] != 'Aggregates']
master_table = master_table[(master_table['GDP_PER_CAPITA'] > 0) | (master_table['CURRENT_GDP'] > 0)]
master_table = master_table.fillna(0)

master_table = pd.melt(master_table, id_vars=['region', 'country'], value_vars=['GDP_PER_CAPITA', 'CURRENT_GDP'], var_name='INDICATOR', value_name='VALUE')
master_table = master_table.set_index(['region', 'country', 'INDICATOR'])
master_table = master_table.sort_index()

master_table

NameError: ignored

**Step 4 : Visualize data with a chart**

In [0]:
table = master_table.reset_index()
gdp_per_capita_per_region = table[table['INDICATOR'] == 'GDP_PER_CAPITA'][['region', 'VALUE']].groupby('region').mean().sort_values('VALUE', ascending=False)
current_gdp_per_region = table[table['INDICATOR'] == 'CURRENT_GDP'][['region', 'VALUE']].groupby('region').mean().sort_values('VALUE', ascending=False)
gdp_per_capita_per_country = table[table['INDICATOR'] == 'GDP_PER_CAPITA'][['country', 'VALUE']].sort_values('VALUE', ascending=False).head(10)
current_gdp_per_country = table[table['INDICATOR'] == 'CURRENT_GDP'][['country', 'VALUE']].sort_values('VALUE', ascending=False).head(10)

data = [
  go.Bar(x=gdp_per_capita_per_region.index, y=gdp_per_capita_per_region['VALUE'], text=gdp_per_capita_per_region['VALUE'], textposition='outside'),
  go.Bar(x=current_gdp_per_region.index, y=current_gdp_per_region['VALUE'], text=current_gdp_per_region['VALUE'], textposition='outside', visible=False),
  go.Bar(x=gdp_per_capita_per_country['country'], y=gdp_per_capita_per_country['VALUE'], text=gdp_per_capita_per_country['VALUE'], textposition='outside', visible=False),
  go.Bar(x=current_gdp_per_country['country'], y=current_gdp_per_country['VALUE'], text=current_gdp_per_country['VALUE'], textposition='outside', visible=False),
]

layout = go.Layout(
  title='Top 10 richest regions & countries',
  margin = dict(t = 60, b = 150),
  updatemenus=list([
    dict(showactive=True, type="buttons", active=0, buttons=[
      {'label': 'GDP / Capita per region', 'method': 'update', 'args': [{'visible': [True, False, False, False]}]},
      {'label': 'Current GDP per region', 'method': 'update', 'args': [{'visible': [False, True, False, False]}]},
      {'label': 'GDP / Capita per country', 'method': 'update', 'args': [{'visible': [False, False, True, False]}]},
      {'label': 'Current GDP per country', 'method': 'update', 'args': [{'visible': [False, False, False, True]}]}
    ])
  ]),
  annotations=[dict(
    text = 'Updated in 2018 from The World Bank',
    showarrow = False,
    xref = 'paper', x = 1,
    yref = 'paper', y = -0.4)]
)

go.Figure(data, layout)

NameError: ignored